# Tensor Network for 2D Classical Ising Model-1

## 4 X 4 Lattice

## API change
* Require Tor10 v 0.3.7 or higher.
* `N_rowrank-->rowrank`.

In [1]:
# import libraries
import Tor10
import copy
import numpy as np
import matplotlib.pyplot as plt

def Tprint(T):
    print(T.Print_diagram())
    print(T)
    
def Tprint_diag(T):
    diag = np.array([T[i,i].item() for i in range(T.bonds[0].dim)])
    print(diag)

def SetZeros(T):
    T.SetElem(np.zeros(T.shape).reshape(-1))
    return

In [2]:
def SVD_n(T):
    M = T.GetBlock()
    UM, S, VtM = M.Svd()
#     T.Print_diagram()
#     UM.Print_diagram()
#     VtM.Print_diagram()
    
    rowrank = T.rowrank
    rank = len(T.labels)
    colrank = rank - rowrank
    
    U_labels = np.concatenate([T.labels[:rowrank],UM.labels[-1:]])
    Vt_labels = np.concatenate([VtM.labels[:1], T.labels[-colrank:]])
    
    U_bonds = np.concatenate([T.bonds[:rowrank],UM.bonds[-1:]])
    Vt_bonds = np.concatenate([VtM.bonds[:1], T.bonds[-colrank:]])

    U = Tor10.UniTensor(bonds=U_bonds, labels=U_labels, rowrank=T.rowrank, name='U') 
    U.PutBlock(UM)
    
    Vt = Tor10.UniTensor(bonds=Vt_bonds, labels=Vt_labels, rowrank=1, name='Vt')
    Vt.PutBlock(VtM)
                          
    return U, S, Vt

In [3]:
def SVD_truncate_n(T, Dcut):
    M = T.GetBlock()
    UM, S, VtM = M.Svd_truncate(Dcut)
#     T.Print_diagram()
#     UM.Print_diagram()
#     VtM.Print_diagram()
    
    rowrank = T.rowrank
    rank = len(T.labels)
    colrank = rank - rowrank
    
    U_labels = np.concatenate([T.labels[:rowrank],UM.labels[-1:]])
    Vt_labels = np.concatenate([VtM.labels[:1], T.labels[-colrank:]])
    
    U_bonds = np.concatenate([T.bonds[:rowrank],UM.bonds[-1:]])
    Vt_bonds = np.concatenate([VtM.bonds[:1], T.bonds[-colrank:]])

    U = Tor10.UniTensor(bonds=U_bonds, labels=U_labels, rowrank=T.rowrank, name='U') 
    U.PutBlock(UM)
    
    Vt = Tor10.UniTensor(bonds=Vt_bonds, labels=Vt_labels, rowrank=1, name='Vt')
    Vt.PutBlock(VtM)
                          
    return U, S, Vt

## Initilize $T$ tensors

In [4]:
def init_T(beta):
    # bond
    bd = Tor10.Bond(2)
    
    # W
    W = Tor10.UniTensor([bd,bd],rowrank=1)
    W.SetName('W')
    W.SetElem([np.sqrt(np.cosh(beta)), np.sqrt(np.sinh(beta)),
               np.sqrt(np.cosh(beta)), -np.sqrt(np.sinh(beta))])
#     Tprint(W)

    # T and Tz
    T = Tor10.UniTensor([bd,bd,bd,bd],rowrank=2, name='T')
    Tz = Tor10.UniTensor([bd,bd,bd,bd],rowrank=2, name='Tz')
    for i in range(2):
        for j in range(2):
            for k in range(2):
                for l in range(2):
                    T[i,j,k,l] = W[0,i].item()*W[0,j].item()*W[0,k].item()*W[0,l].item() + W[1,i].item()*W[1,j].item()*W[1,k].item()*W[1,l].item()     
                    Tz[i,j,k,l] = W[0,i].item()*W[0,j].item()*W[0,k].item()*W[0,l].item()- W[1,i].item()*W[1,j].item()*W[1,k].item()*W[1,l].item()     
#     Tprint(T.GetBlock())
#     Tprint(Tz.GetBlock())

    return T, Tz

In [5]:
T, Tz = init_T(1)
# Tprint(T)
# Tprint(Tz)
# Tprint(T.GetBlock())
# Tprint(Tz.GetBlock())

## Row to Row merger: Merger T_up and T_dn

### Create MM
* Contract $T_{up,dn}$ and $T^t_{up,dn}$ to get $MM_{up,dn}$.
* Contract $MM_{up,dn}$ to get $MM$.

In [6]:
T_up = copy.deepcopy(T)
T_up.SetName('T_up')
T_upt = T_up.Whole_transpose()
T_upt.SetName('T_upt')
T_upt.SetLabels([2,13,10,11])
T_up.Print_diagram()
T_upt.Print_diagram()

MM_up = Tor10.Contract(T_up, T_upt)
MM_up.SetName('MM_up')
MM_up.Print_diagram()

MM_dn = copy.deepcopy(MM_up)
MM_dn.SetName('MM_dn')
MM_dn.SetLabels([13,101,0,10,3,111])
MM_dn.Print_diagram()

MM = Tor10.Contract(MM_up, MM_dn)
MM.SetName('MM')
MM.Print_diagram()

-----------------------
tensor Name : T_up
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           |             |     
     1 ____| 2         2 |____ 3  
           \             /     
            -------------      
-----------------------
tensor Name : T_upt
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     2 ____| 2         2 |____ 10 
           |             |     
    13 ____| 2         2 |____ 11 
           \             /     
            -------------      
-----------------------
tensor Name : MM_up
tensor Rank : 6
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 2         2 |____ 3  
           |             |     
     1 ____| 2         2 |____ 10 
         

### SVD on MM to get $U^L$

In [7]:
U, S, Ut = SVD_n(MM)
U.SetName('U')
Ut.SetName('Ut')
U.Print_diagram()
Ut.Print_diagram()

-----------------------
tensor Name : U
tensor Rank : 3
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     1 ____| 2         4 |____ -1 
           |             |     
   101 ____| 2           |        
           \             /     
            -------------      
-----------------------
tensor Name : Ut
tensor Rank : 3
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
    -2 ____| 4         2 |____ 11 
           |             |     
           |           2 |____ 111
           \             /     
            -------------      


### HOTRG T_up and T_dn to get T_new
* Merge T_up and T_dn to be T_updn.
* $T_{new} = U^{Lt} T_{updn} U^L$.

In [8]:
T_up.SetLabels([0,1,2,-3])
T_dn = copy.deepcopy(T)
T_dn.SetName('T_dn')
T_dn.SetLabels([-3,11,12,3])
T_up.Print_diagram()
T_dn.Print_diagram()
T_updn = Tor10.Contract(T_up, T_dn)
T_updn.SetName('T_updn')
T_updn.SetLabels([0,-1,-11,-2,-12,3])
T_updn.Print_diagram()

U, S, Ut = SVD_n(MM)
U.SetName('U')
U.SetLabels([-2,-12,2])
Ut.SetName('Ut')
Ut.SetLabels([1,-1,-11])
U.Print_diagram()
Ut.Print_diagram()

T_new = Tor10.Contract(Tor10.Contract(Ut, T_updn), U)
T_new.SetName('T_new')
T_new.Print_diagram()
T_new.Permute([0,1,2,3], by_label=True)
T_new.Print_diagram()

-----------------------
tensor Name : T_up
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           |             |     
     1 ____| 2         2 |____ -3 
           \             /     
            -------------      
-----------------------
tensor Name : T_dn
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
    -3 ____| 2         2 |____ 12 
           |             |     
    11 ____| 2         2 |____ 3  
           \             /     
            -------------      
-----------------------
tensor Name : T_updn
tensor Rank : 6
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 2         2 |____ -2 
           |             |     
    -1 ____| 2         2 |____ -12
         

In [9]:
TT = T_new.Permute([1,3,0,2], by_label=True, rowrank=2)
TT.Print_diagram()
TT.SetLabels([0,1,2,3])
TT.Print_diagram()

-----------------------
tensor Name : T_new
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     1 ____| 4         2 |____ 0  
           |             |     
     3 ____| 2         4 |____ 2  
           \             /     
            -------------      
-----------------------
tensor Name : T_new
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 4         2 |____ 2  
           |             |     
     1 ____| 2         4 |____ 3  
           \             /     
            -------------      


In [10]:
T_up = copy.deepcopy(TT)
T_up.SetName('T_up')
T_upt = T_up.Whole_transpose()
T_upt.SetName('T_upt')
T_upt.SetLabels([2,13,10,11])
T_up.Print_diagram()
T_upt.Print_diagram()

MM_up = Tor10.Contract(T_up, T_upt)
MM_up.SetName('MM_up')
MM_up.Print_diagram()

MM_dn = copy.deepcopy(MM_up)
MM_dn.SetName('MM_dn')
MM_dn.SetLabels([13,101,0,10,3,111])
MM_dn.Print_diagram()

MM = Tor10.Contract(MM_up, MM_dn)
MM.SetName('MM')
MM.Print_diagram()

-----------------------
tensor Name : T_up
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 4         2 |____ 2  
           |             |     
     1 ____| 2         4 |____ 3  
           \             /     
            -------------      
-----------------------
tensor Name : T_upt
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     2 ____| 2         4 |____ 10 
           |             |     
    13 ____| 4         2 |____ 11 
           \             /     
            -------------      
-----------------------
tensor Name : MM_up
tensor Rank : 6
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 4         4 |____ 3  
           |             |     
     1 ____| 2         4 |____ 10 
         

In [11]:
T_up.SetLabels([0,1,2,-3])
T_dn = copy.deepcopy(TT)
T_dn.SetName('T_dn')
T_dn.SetLabels([-3,11,12,3])
T_up.Print_diagram()
T_dn.Print_diagram()
T_updn = Tor10.Contract(T_up, T_dn)
T_updn.SetName('T_updn')
T_updn.SetLabels([0,-1,-11,-2,-12,3])
T_updn.Print_diagram()

U, S, Ut = SVD_n(MM)
U.SetName('U')
U.SetLabels([-2,-12,2])
Ut.SetName('Ut')
Ut.SetLabels([1,-1,-11])
U.Print_diagram()
Ut.Print_diagram()

TT_new = Tor10.Contract(Tor10.Contract(Ut, T_updn), U)
TT_new.SetName('T_new')
TT_new.Print_diagram()
TT_new.Permute([0,1,2,3], by_label=True)
TT_new.Print_diagram()

-----------------------
tensor Name : T_up
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 4         2 |____ 2  
           |             |     
     1 ____| 2         4 |____ -3 
           \             /     
            -------------      
-----------------------
tensor Name : T_dn
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
    -3 ____| 4         2 |____ 12 
           |             |     
    11 ____| 2         4 |____ 3  
           \             /     
            -------------      
-----------------------
tensor Name : T_updn
tensor Rank : 6
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 4         2 |____ -2 
           |             |     
    -1 ____| 2         2 |____ -12
         